<a href="https://colab.research.google.com/github/UiinKim/UiinKim/blob/main/KoGPT_2_%EC%B1%97%EB%B4%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import torch
import torchvision
import torch.nn

In [4]:
!pip install --upgrade torch torchvision

In [5]:
torch.cuda.is_available()

True

In [6]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [7]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='<s>', eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.9.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'lm_head.weight', 'transformer.h.3.attn.masked_bias', 'transformer.h.10.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [8]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-'*10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

0
1
3
----------
</s>
<usr>
<pad>
<sys>


In [9]:
import pandas as pd
import tqdm
import urllib.request

In [10]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data=pd.read_csv('ChatBotData.csv')

In [11]:
len(train_data)

11823

In [12]:
batch_size=32

In [13]:
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token=[tokenizer.bos_token_id]
    eos_token=[tokenizer.eos_token_id]
    sent=tokenizer.encode(''+question+''+answer)
    yield bos_token+sent+eos_token
    #return은 list를 반환하지만 yield는 반복문을 통해 제너레이터(필요할때마다 즉석해서 하나씩 만들어낼 수 있는 객체)를 반환

In [14]:
dataset=tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

In [15]:
dataset=dataset.padded_batch(batch_size=batch_size, padded_shapes=(None, ), padding_values=tokenizer.pad_token_id)

In [16]:
for batch in dataset:
  print(batch)
  break

tf.Tensor(
[[    0  9349  7888   739  7318   376 25000  6824  9108  9028  7098 25856
      1     3     3     3     3     3     3     3     3     3     3     3
      3     3     3     3]
 [    0  9020  8263  7497 10192 11615  8210  8006 11567  8711  9535  7483
  12521     1     3     3     3     3     3     3     3     3     3     3
      3     3     3     3]
 [    0  9085  7597   395  8149 10624  7397 24224 13358  7182  8030 19138
  16899  9677  8234   389     1     3     3     3     3     3     3     3
      3     3     3     3]
 [    0  9085  7597   395  8149  9465 10624  7397 24224 13358  7182  8030
  19138 16899  9677  8234   389     1     3     3     3     3     3     3
      3     3     3     3]
 [    0  9943   422   418  9327  8702  7098  7141 16912 18328  8671  7415
   8263  8234   389     1     3     3     3     3     3     3     3     3
      3     3     3     3]
 [    0  9815   410 21249 10174  6824  8210  8006 16146 11056 11594 10137
  10556  9266  8711 25856     1     3   

In [17]:
tokenizer.decode(batch[0])

'<s> 12시 땡!하루가 또 가네요.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [18]:
print(batch[0])

tf.Tensor(
[    0  9349  7888   739  7318   376 25000  6824  9108  9028  7098 25856
     1     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3], shape=(28,), dtype=int32)


In [19]:
print(tokenizer.encode(' 12시 땡! 하루가 또 가네요.'))

[9349, 7888, 739, 7318, 376, 12557, 6824, 9108, 9028, 7098, 25856]


In [20]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [21]:
steps=len(train_data)//batch_size+1
#단계를 총 길이에서 묶음 개수 만큼 나눈다.
print(steps)

370


In [24]:
EPOCHS = 3

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.tqdm_notebook(dataset, total=steps):
      with tf.GradientTape() as tape:
          result = model(batch, labels=batch)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)

      grads = tape.gradient(batch_loss, model.trainable_variables)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

<ipython-input-24-7f8b2a38ff99>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm.tqdm_notebook(dataset, total=steps):


  0%|          | 0/370 [00:00<?, ?it/s]

InvalidArgumentError: ignored

In [ ]:
text='오늘도 좋은 하루!'

In [ ]:
sent=''+text+''

In [ ]:
input_ids=[tokenizer.bos_token_id]+tokenizer.encode(sent)
input_ids=tf.convert_to_tensor([input_ids])

In [ ]:
output=model.generate(input_ids, max_length=50, early_stopping=True, eos_toekn_id=tokenizer.eos_token_id)

In [ ]:
decoded_sentence=tokenizer.decode(output[0].numpy().tolist())

In [ ]:
decoded_sentence.split(' ')[1].replace('','')

In [ ]:
output=model.generate(input_ids, max_length=50, do_sample=True, top_k=10)
tokenizer.decode(output[0].numpy().tolist())

In [ ]:
def return_answer_by_chatbot(user_text):
  sent=''+user_text+''
  input_ids=[tokenizer.bos_token_id]+tokenizer.encode(sent)
  input_ids=tf.convert_to_tensor([input_ids])
  output=model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
  sentence=tokenizer.decode(output[0].numpy().tolist())
  chatbot_response=sentence.split(' ')[1].replace('','')
  return chatbot_response

In [ ]:
return_answer_by_chatbot('안녕! 반가워~')

In [ ]:
return_answer_by_chatbot('너는 누구야?')

In [ ]:
return_answer_by_chatbot('사랑해')

In [ ]:
return_answer_by_chatbot('나랑 영화보자')

In [ ]:
return_answer_by_chatbot('너무 심심한데 나랑 놀자')

In [ ]:
return_answer_by_chatbot('영화 해리포터 재밌어?')

In [ ]:
return_answer_by_chatbot('너 딥 러닝 잘해?')

In [ ]:
return_answer_by_chatbot('너 취했어?')

In [ ]:
return_answer_by_chatbot('커피 한 잔 할까?')